<a href="https://colab.research.google.com/github/mark-6941/AAAAAAAA/blob/master/Quantam_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import paddle
try:
    import paddle_quantum
    print("Paddle Quantum 導入成功")
except ImportError:
    print("請先執行 !pip install paddle-quantum")

import numpy as np
from paddle.vision.datasets import MNIST
from paddle.vision.transforms import ToTensor

# 0. 設備設定
# 優先嘗試 NPU，若失敗則回退至 CPU
try:
    paddle.device.set_custom_device('custom_npu')
    print("目前使用設備: custom_npu")
except:
    paddle.set_device('cpu')
    print("環境不支援 custom_npu，已切換至 CPU")

# 1. 定義數據集與 DataLoader
transform = ToTensor()
train_dataset = MNIST(mode='train', transform=transform)

train_loader = paddle.io.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True
)

# 2. 定義模型 (經典部分範例)
model = paddle.nn.Sequential(
    paddle.nn.Flatten(),
    paddle.nn.Linear(784, 10)
)

# 3. 執行數據迭代測試
print("開始測試數據讀取...")
for batch_id, (data, label) in enumerate(train_loader):
    # data 自動會在 set_custom_device 指定的設備上
    x_batch = paddle.flatten(data, start_axis=1)

    # 模型預測
    y_pred = model(x_batch)

    if batch_id == 0:
        print(f"成功處理第一組 Batch!")
        print(f"數據形狀: {x_batch.shape}, 預測結果形狀: {y_pred.shape}")
        break

In [4]:
import paddle
import numpy as np

# --- 模擬設備設定 ---
try:
    paddle.device.set_custom_device('custom_npu')
    print("🔋 雲端儲能系統已連接至 NPU 加速器")
except:
    paddle.set_device('cpu')
    print("⚠️ 未偵測到 NPU，使用 CPU 運行儲能模擬")

# --- 1. 儲能數據產生器 (模擬真實感測器數據) ---
# 假設欄位：[電壓V, 電流I, 溫度T, 內部阻抗R]
def get_ess_sensor_data(batch_size=32):
    # 這裡模擬儲能櫃上傳的實時數據
    x = np.random.rand(batch_size, 4).astype('float32')
    # 標籤：SOH (State of Health)
    y = np.random.uniform(0.7, 1.0, (batch_size, 1)).astype('float32')
    return paddle.to_tensor(x), paddle.to_tensor(y)

# --- 2. 量子增強儲能預測模型 (解耦設計) ---
class ESSQuantumInference(paddle.nn.Layer):
    def __init__(self):
        super().__init__()
        # 經典編碼層：將感測器數據映射到高維空間 (類似量子編碼)
        self.encoder = paddle.nn.Linear(4, 16)

        # 模擬量子變分電路 (VQC) 的作用
        # 這裡利用多層非線性變換模擬量子糾纏的效果
        self.quantum_proxy = paddle.nn.Sequential(
            paddle.nn.Tanh(),
            paddle.nn.Linear(16, 16),
            paddle.nn.ReLU()
        )

        # 輸出層：預測雲端剩餘壽命或 SOC
        self.decoder = paddle.nn.Linear(16, 1)

    def forward(self, x):
        encoded = self.encoder(x)
        # 模擬量子干涉過程
        q_features = self.quantum_proxy(encoded)
        return self.decoder(q_features)
    def forward(self, F):
        # 前向传播逻辑
        x = F.relu(self.fc1(x))  # 第一层 + ReLU 激活
        x = self.fc2(x)          # 第二层
        return x

    def __init__(self,nn):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)  # 第一层：输入特征10个，输出特征5个
        self.relu = nn.ReLU()        # 非线性激活函数ReLU
        self.fc2 = nn.Linear(5, 1)   # 第二层：输入特征5个，输出特征1个

    def forward(self, x):
        x = self.fc1(x)  # 数据通过第一层
        x = self.relu(x) # 应用ReLU激活函数
        x = self.fc2(x)  # 数据通过第二层
        return x



# --- 3. 測試執行 ---
model = ESSQuantumInference()
x, y = get_ess_sensor_data()
pred = model(x)

print(f"--- 儲能系統預測完成 ---")
print(f"輸入特徵 (第一筆數據): {x[0].numpy()}")
print(f"預測 SOH (電池健康度): {pred[0].item():.10f}")

ModuleNotFoundError: No module named 'paddle'

In [7]:

import snap7
from snap7.util import get_real  # <--- 這樣才能正確導入函式
#from snap7.util
#import get_real  # 必須明確導入，否則會紅字報錯
import torch
import torch.nn as nn
import numpy
#from pymodbus.payload import BinaryPayloadDecoder
#from pymodbus.framer import Endian
#from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.framer import Endian  # 關鍵修正點
from pymodbus.client import ModbusTcpClient
# from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.constants import Endian
import time


def run_ai_with_ti_chip():
# 1. 建立連線實例 (請填入實際的 PLC IP)
    client = ModbusTcpClient('192.168.0.10', port=502)

    # 2. 嘗試連線
    connection = client.connect()
    if not connection:
        print("連線失敗：請檢查 PLC IP 與網路設定")
        return

    try:
        # 3. 讀取 4 個 Float (每個佔 2 個寄存器，總共讀 8 個)
        # address=0 是起始位址，count=8 代表讀取 8 個 16-bit 寄存器
        result = client.read_holding_registers(address=0, count=8, slave=1)

        if result.isError():
            print("讀取失敗：Modbus 報錯")
            return

        # 4. 使用解碼器處理 16-bit 轉 32-bit Float
        # 根據您的硬體調整 Endian (大端或小端)
        decoder = BinaryPayloadDecoder.fromRegisters(
            result.registers,
            byteorder=Endian.BIG,
            wordorder=Endian.BIG
        )

        # 依序讀取 4 個浮點數
        floats = [decoder.decode_32bit_float() for _ in range(4)]
        print(f"讀取成功：{floats}")
        return floats

    except Exception as e:
        print(f"執行出錯: {e}")
    finally:
        # 5. 確保關閉連線
        client.close()

        # 使用解碼器 (對應圖片 3 的導入內容)
        decoder = BinaryPayloadDecoder.fromRegisters(
            result.registers,
            byteorder=Endian.Big,
            wordorder=Endian.Big
        )

        # 依照 0, 4, 8, 12 的邏輯（在 Modbus 中是 Offset 0, 2, 4, 6 寄存器）
        v = decoder.decode_32bit_float()
        i = decoder.decode_32bit_float()
        t = decoder.decode_32bit_float()
        r = decoder.decode_32bit_float()



# PLC 连接参数
PLC_IP = "192.168.0.10"  # PLC IP 地址
PLC_PORT = 502           # Modbus TCP 默认端口

client = ModbusTcpClient(PLC_IP, port=PLC_PORT)
if not client.connect():
    print("❌ 无法连接到 PLC，请检查网络和 IP 设置")
    exit(1)

# 初始化 PLC 通訊
plc = snap7.client.Client()

try:
    # 這裡的 IP 需與您的 Siemens PLC (S7-1200/1500) 一致
    # Rack 0, Slot 1 是 S7-1200/1500 的標準設定
    plc.connect('192.168.0.1', 0, 1)
    print("✅ 已成功連接 Siemens PLC")
except Exception as e:
    print(f"❌ 連接失敗: {e}")

# 读取保持寄存器（Holding Registers），假设地址 0 开始，读取 10 个
    rr = client.read_holding_registers(address=0, count=10, unit=1)

    if isinstance(rr, ModbusIOException) or rr.isError():
        print("❌ 读取 PLC 数据失败")
    else:
        # 转换为 NumPy 数组
        data = np.array(rr.registers, dtype=np.int16)
        print("📥 原始数据:", data)

def run_ai_with_plc():
    if not plc.get_connected():
        print("PLC 未連接")
        return

    # A. 讀取 PLC 訊號 (DB1, 起始位址 0, 長度 16 bytes)
    # 這 16 bytes 將被解析為 4 個 32-bit Float
    try:
        data = plc.db_read(1, 0, 16)

        # 使用 snap7.util.get_real 正確解析
        v = get_real(data, 0)   # Offset 0: 電壓
        i = get_real(data, 4)   # Offset 4: 電流
        t = get_real(data, 8)   # Offset 8: 溫度
        r = get_real(data, 12)  # Offset 12: 內阻

        print(f"📡 讀取晶片訊號 -> V: {v:.2f}, I: {i:.2f}, T: {t:.2f}, R: {r:.2f}")

        # 這裡可以接上您之前的 AI 模型進行推理...
        # input_tensor = torch.tensor([[v, i, t, r]], dtype=torch.float32)

    except Exception as e:
        print(f"數據讀取失敗: {e}")

# 執行測試
if plc.get_connected():
    run_ai_with_plc()

ImportError: cannot import name 'Endian' from 'pymodbus.framer' (/usr/local/lib/python3.12/dist-packages/pymodbus/framer/__init__.py)

In [1]:

import snap7
from snap7.util import get_real  # <--- 這樣才能正確導入函式
#from snap7.util
#import get_real  # 必須明確導入，否則會紅字報錯
import torch
import torch.nn as nn
import numpy
#from pymodbus.payload import BinaryPayloadDecoder
#from pymodbus.framer import Endian
#from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.framer import Endian  # 關鍵修正點
from pymodbus.client import ModbusTcpClient
# from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.constants import Endian
import time


def run_ai_with_ti_chip():
# 1. 建立連線實例 (請填入實際的 PLC IP)
    client = ModbusTcpClient('192.168.0.10', port=502)

    # 2. 嘗試連線
    connection = client.connect()
    if not connection:
        print("連線失敗：請檢查 PLC IP 與網路設定")
        return

    try:
        # 3. 讀取 4 個 Float (每個佔 2 個寄存器，總共讀 8 個)
        # address=0 是起始位址，count=8 代表讀取 8 個 16-bit 寄存器
        result = client.read_holding_registers(address=0, count=8, slave=1)

        if result.isError():
            print("讀取失敗：Modbus 報錯")
            return

        # 4. 使用解碼器處理 16-bit 轉 32-bit Float
        # 根據您的硬體調整 Endian (大端或小端)
        decoder = BinaryPayloadDecoder.fromRegisters(
            result.registers,
            byteorder=Endian.BIG,
            wordorder=Endian.BIG
        )

        # 依序讀取 4 個浮點數
        floats = [decoder.decode_32bit_float() for _ in range(4)]
        print(f"讀取成功：{floats}")
        return floats

    except Exception as e:
        print(f"執行出錯: {e}")
    finally:
        # 5. 確保關閉連線
        client.close()

        # 使用解碼器 (對應圖片 3 的導入內容)
        decoder = BinaryPayloadDecoder.fromRegisters(
            result.registers,
            byteorder=Endian.Big,
            wordorder=Endian.Big
        )

        # 依照 0, 4, 8, 12 的邏輯（在 Modbus 中是 Offset 0, 2, 4, 6 寄存器）
        v = decoder.decode_32bit_float()
        i = decoder.decode_32bit_float()
        t = decoder.decode_32bit_float()
        r = decoder.decode_32bit_float()



# PLC 连接参数
PLC_IP = "192.168.0.10"  # PLC IP 地址
PLC_PORT = 502           # Modbus TCP 默认端口

client = ModbusTcpClient(PLC_IP, port=PLC_PORT)
if not client.connect():
    print("❌ 无法连接到 PLC，请检查网络和 IP 设置")
    exit(1)

# 初始化 PLC 通訊
plc = snap7.client.Client()

try:
    # 這裡的 IP 需與您的 Siemens PLC (S7-1200/1500) 一致
    # Rack 0, Slot 1 是 S7-1200/1500 的標準設定
    plc.connect('192.168.0.1', 0, 1)
    print("✅ 已成功連接 Siemens PLC")
except Exception as e:
    print(f"❌ 連接失敗: {e}")

# 读取保持寄存器（Holding Registers），假设地址 0 开始，读取 10 个
    rr = client.read_holding_registers(address=0, count=10, unit=1)

    if isinstance(rr, ModbusIOException) or rr.isError():
        print("❌ 读取 PLC 数据失败")
    else:
        # 转换为 NumPy 数组
        data = np.array(rr.registers, dtype=np.int16)
        print("📥 原始数据:", data)

def run_ai_with_plc():
    if not plc.get_connected():
        print("PLC 未連接")
        return

    # A. 讀取 PLC 訊號 (DB1, 起始位址 0, 長度 16 bytes)
    # 這 16 bytes 將被解析為 4 個 32-bit Float
    try:
        data = plc.db_read(1, 0, 16)

        # 使用 snap7.util.get_real 正確解析
        v = get_real(data, 0)   # Offset 0: 電壓
        i = get_real(data, 4)   # Offset 4: 電流
        t = get_real(data, 8)   # Offset 8: 溫度
        r = get_real(data, 12)  # Offset 12: 內阻

        print(f"📡 讀取晶片訊號 -> V: {v:.2f}, I: {i:.2f}, T: {t:.2f}, R: {r:.2f}")

        # 這裡可以接上您之前的 AI 模型進行推理...
        # input_tensor = torch.tensor([[v, i, t, r]], dtype=torch.float32)

    except Exception as e:
        print(f"數據讀取失敗: {e}")

# 執行測試
if plc.get_connected():
    run_ai_with_plc()

ModuleNotFoundError: No module named 'snap7'

In [3]:
# 安裝 Modbus 與 Snap7 的 Python 套件
!pip install pymodbus python-snap7 numpy torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.5/169.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 19.5 MB/s eta 0:00:00


In [6]:
# 1. Modbus TCP 通訊相關 (用於 ModbusTcpClient)
from pymodbus.client import ModbusTcpClient
from pymodbus.exceptions import ModbusIOException
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder

# 2. Siemens S7 通訊相關 (用於 snap7)
import snap7
from snap7.util import get_real

# 3. 數據處理與 AI 相關 (用於 np.array 與 torch)
import numpy as np
import torch  # 如果後續有用到 AI 推理部分

# 4. 其他基礎工具
import sys

ImportError: cannot import name 'Endian' from 'pymodbus.constants' (/usr/local/lib/python3.12/dist-packages/pymodbus/constants.py)

ModuleNotFoundError: No module named 'snap7'

In [ ]:
# 安裝 Modbus 與 S7 通訊所需的套件
!pip install pymodbus snap7

  Using cached pymodbus-3.12.1-py3-none-any.whl.metadata (17 kB)
ERROR: Could not find a version that satisfies the requirement snap7 (from versions: none)
ERROR: No matching distribution found for snap7


In [ ]:
!sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,378 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,910 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd

In [ ]:
!sudo apt-get install -y libsnap7-dev build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libsnap7-dev


In [ ]:
!pip install python-snap7 pymodbus

  Using cached pymodbus-3.12.1-py3-none-any.whl.metadata (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.1/145.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.5/169.5 kB 18.5 MB/s eta 0:00:00


In [ ]:
pip install pymodbus snap7

  Using cached pymodbus-3.12.1-py3-none-any.whl.metadata (17 kB)
ERROR: Could not find a version that satisfies the requirement snap7 (from versions: none)
ERROR: No matching distribution found for snap7


In [ ]:
import torch
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # 1. 定義網路層
        # 假設輸入維度為 4 (ESS 感測器數據)，輸出維度為 1 (SOH 健康度)
        self.fc1 = nn.Linear(4, 16)   # 第一層全連接
        self.relu = nn.ReLU()         # 激活函數
        self.fc2 = nn.Linear(16, 1)   # 輸出層

    def forward(self, x):
        # 2. 定義前向傳播過程
        x = self.fc1(x)
        x = self.relu(x)
        output = self.fc2(x)          # 計算 output
        return output                 # 現在 output 已經定義，可以回傳了

# --- 測試模型 ---
model = SimpleNet()
# 模擬一組儲能數據 [Batch_size=1, Features=4]
mock_input = torch.randn(1, 4)
prediction = model(mock_input)

print(f"雲端儲能系統預測結果: {prediction.item():.4f}")

In [ ]:
import torch
import time

# 1. 初始化參數 (對應 C++ 中的矩陣維度)
m =1024 # batch_size * seq_len
k = 768 # hidden_dim
n = 512 # head_num * size_per_head

# 確認 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 分配與初始化張量 (對應 C++ 的 cudaMalloc 與初始化)
# 我們使用 float32 (对应 CUDA_R_32F)
# 權重矩陣 W_Q, W_K, W_V
attr_kernel_Q = torch.randn(k, n, device=device)
attr_kernel_K = torch.randn(k, n, device=device)
attr_kernel_V = torch.randn(k, n, device=device)

# 輸入張量 (Hidden States)
from_tensor = torch.randn(m, k, device=device)
to_tensor = from_tensor # 在 Self-attention 中通常是同一個輸入

# 3. 執行 Q, K, V 的矩陣乘法 (對應 cublasGemmEx)
# 公式: Output = Input * Weight
# 注意: PyTorch 的矩陣乘法會自動調用底層優化過的 cuBLAS GEMM

# 開始計時
torch.cuda.synchronize() # 確保 GPU 前置作業完成
start_time = time.time()

# 計算 Query: Q = Hidden_States * W_Q
query_buf_ = torch.matmul(from_tensor, attr_kernel_Q)

# 計算 Key: K = Hidden_States * W_K
key_buf_ = torch.matmul(to_tensor, attr_kernel_K)

# 計算 Value: V = Hidden_States * W_V
value_buf_ = torch.matmul(to_tensor, attr_kernel_V)

# 等待 GPU 運算結束
torch.cuda.synchronize()
end_time = time.time()

# 4. 輸出結果
print("-" * 30)
print("Q, K, V GEMM 計算完成 (PyTorch/cuBLAS)！")
print(f"運算耗時: {(end_time - start_time) * 1000:.4f} ms")
print(f"Query Buffer Shape: {query_buf_.shape}")
print(f"Key Buffer Shape:   {key_buf_.shape}")
print(f"Value Buffer Shape: {value_buf_.shape}")
print("-" * 30)

# 顯示前幾個數值驗證 (對應 C++ 的 std::cout)
print("Query 前五個數值範例:")
print(query_buf_[0, :5])

Using device: cuda
------------------------------
Q, K, V GEMM 計算完成 (PyTorch/cuBLAS)！
運算耗時: 2.7637 ms
Query Buffer Shape: torch.Size([1024, 512])
Key Buffer Shape:   torch.Size([1024, 512])
Value Buffer Shape: torch.Size([1024, 512])
------------------------------
Query 前五個數值範例:
tensor([  9.6516,  41.7510,  50.4923,  17.8632, -33.6008], device='cuda:0')


In [ ]:
pip install python-snap7 pymodbus boto3 torch paddlepaddle

In [ ]:
pip install python-snap7

In [ ]:
pip install python-snap7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.1/145.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
import paddle
import numpy as np

# 模擬儲能數據獲取
def get_ess_vqc_data():
    # 假設從電池管理系統 (BMS) 獲取數據
    # [電壓, 電流, 溫度, 內阻] -> 標籤: 1 (需要均衡充電), 0 (不需處理)
    ess_raw = np.array([
        [3.6, 0.5, 25, 12, 0],
        [3.2, 0.8, 45, 15, 1],
        # ... 更多實時數據
    ])
    data = ess_raw[:, :4]  # 前 4 位是感測器特徵
    label = ess_raw[:, 4]  # 最後一位是控制決策
    return data, label

# 在 run2 裡面整合
def run_ess_optimization():
    # model = QModel(4, ...) # 4 個輸入對應 4 個感測器
    # ... 訓練過程 ...
    # 當 result > 0.5 時，觸發儲能櫃均衡電路開關
    if result.item() > 0.5:
        print("警告：偵測到電池不平衡，啟動儲能均衡補償")

/usr/local/lib/python3.12/dist-packages/paddle/utils/cpp_extension/extension_utils.py:712: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [ ]:
# 嘗試從 GitHub 源碼安裝，這有機會繞過 PyPI 的版本鎖定
!pip install git+https://github.com/PaddlePaddle/Quantum.git

  Cloning https://github.com/PaddlePaddle/Quantum.git to /tmp/pip-req-build-kr3pl2it
  Running command git clone --filter=blob:none --quiet https://github.com/PaddlePaddle/Quantum.git /tmp/pip-req-build-kr3pl2it
  Resolved https://github.com/PaddlePaddle/Quantum.git to commit 760c787f29e18db77ec6fba5e91da6166ae6599e
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of paddle-quantum to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement paddlepaddle<=2.3.0,>=2.2.0 (from paddle-quantum) (from versions: 2.6.2, 3.0.0b1, 3.0.0b2, 3.0.0rc0, 3.0.0rc1, 3.0.0, 3.1.0, 3.1.1, 3.2.0, 3.2.1, 3.2.2, 3.3.0)
ERROR: No matching distribution found for paddlepaddle<=2.3.0,>=2.2.0


In [ ]:
# 安裝 Paddle Quantum 套件
!pip install paddle-quantum -i https://mirror.baidu.com/pypi/simple

In [ ]:
import sys
print(sys.version)

In [ ]:
!pip install paddle-quantum -i https://mirror.baidu.com/pypi/simple

In [ ]:
pip install paddlepaddle -U

In [ ]:
pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple


In [ ]:
!docker build -t paddle-quantum-npu .

In [ ]:
RUN pip install --no-cache-dir paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple && \
    pip install --no-cache-dir "protobuf<3.21" --force-reinstall && \
    pip install --no-cache-dir paddle-quantum

In [ ]:
# 使用 %%writefile 將內容寫入實體檔案，而非執行它
%%writefile Dockerfile

FROM python:3.12-slim

# 安裝基礎系統工具
RUN apt-get update && apt-get install -y \
    ccache \
    curl \
    && rm -rf /var/lib/apt/lists/*

WORKDIR /app

# 依照正確順序安裝套件，解決您之前的 Protobuf 衝突
RUN pip install --no-cache-dir paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple && \
    pip install --no-cache-dir "protobuf<3.21" --force-reinstall && \
    pip install --no-cache-dir paddle-quantum

# 解決 TypeError 的關鍵環境變數
ENV PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

COPY . /app
CMD ["python", "train.py"]

In [ ]:
import paddle
import paddle_quantum

# 檢查是否有 GPU，否則用 CPU
device_str = 'gpu' if paddle.is_compiled_with_cuda() else 'cpu'
paddle.set_device(device_str)
print(f"✅ 已切換至設備: {device_str}")


In [ ]:
import paddle
# 執行這行看輸出結果
print("目前可用的自定義設備:", paddle.device.get_all_custom_device_type())

In [ ]:
import paddle
import os

# 1. 檢查硬體文件是否存在於環境中
if os.path.exists('/dev/galcore') or os.path.exists('/dev/davinci0'):
    print("✅ 系統層：硬體設備文件已掛載")
else:
    print("❌ 系統層：找不到設備文件，請檢查 Docker/K8s 掛載設定")

if os.
# 2. 檢查 Paddle 驅動支持
supported_devices = paddle.device.get_all_custom_device_type()
print(f"Paddle 偵測到的硬體類型: {supported_devices}")

# 3. 嘗試指派
try:
    if 'npu' in supported_devices:
        paddle.set_device('npu')
        print("🚀 成功掛載並切換至 NPU 運算模式")
    else:
        paddle.set_device('cpu')
        print("⚠️ 未偵測到 NPU 插件，目前使用 CPU 運行")
except Exception as e:
    print(f"🚨 掛載出錯: {e}")

In [ ]:
# 1. 解決 Protobuf 相容性問題
!pip install "protobuf<3.21" --force-reinstall

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import paddle
import paddle_quantum
print("✅ 成功解決衝突並載入量槳")

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import paddle
import paddle_quantum

In [ ]:
device = 'gpu' if paddle.is_compiled_with_cuda() else 'cpu'
paddle.set_device(device)
print(f"✅ 目前使用 {device} 進行模擬運算")

In [ ]:
# 1. 降級 Protobuf 到相容版本 (3.20.x)
!pip install "protobuf<3.21" --force-reinstall

In [ ]:
import sys
!{sys.executable} -m pip install paddlepaddle paddle-quantum -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install paddle-quantum

In [ ]:
!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple